<a href="https://colab.research.google.com/github/lapatradaa/MCTS/blob/main/UM_Game_Playing_Strength_of_MCTS_Variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train = pd.read_csv('/content/drive/MyDrive/MCTS/train.csv')
test = pd.read_csv('/content/drive/MyDrive/MCTS/test.csv')

In [5]:
train.head(5)

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333


In [6]:
# prompt: print number of unique values in all columns

for column in train.columns:
  print(f"{column}: {train[column].nunique()}")


Id: 233234
GameRulesetName: 1377
agent1: 72
agent2: 72
Properties: 1
Format: 1
Time: 1
Discrete: 1
Realtime: 1
Turns: 1
Alternating: 1
Simultaneous: 1
Stochastic: 2
HiddenInformation: 1
Match: 1
Asymmetric: 2
AsymmetricRules: 1
AsymmetricPlayRules: 1
AsymmetricEndRules: 1
AsymmetricForces: 2
AsymmetricSetup: 1
AsymmetricPiecesType: 2
Players: 1
NumPlayers: 1
Simulation: 1
Solitaire: 1
TwoPlayer: 1
Multiplayer: 1
PlayersWithDirections: 2
Cooperation: 2
Team: 2
Coalition: 1
Puzzle: 1
DeductionPuzzle: 1
PlanningPuzzle: 1
Equipment: 1
Container: 1
Board: 1
Shape: 2
SquareShape: 2
HexShape: 2
TriangleShape: 2
DiamondShape: 2
RectangleShape: 2
SpiralShape: 2
CircleShape: 2
PrismShape: 1
StarShape: 2
ParallelogramShape: 1
SquarePyramidalShape: 2
RectanglePyramidalShape: 1
RegularShape: 2
PolygonShape: 2
TargetShape: 1
Tiling: 2
SquareTiling: 2
HexTiling: 2
TriangleTiling: 2
BrickTiling: 1
SemiRegularTiling: 2
CelticTiling: 1
MorrisTiling: 2
QuadHexTiling: 1
CircleTiling: 2
ConcentricTiling: 2

In [7]:
# prompt: drop all columns with no unique values

for column in train.columns:
  if train[column].nunique() == 1:
    train = train.drop(column, axis=1)


In [8]:
# prompt: print the number of columns and the number of unqiue values of each column

print(f"Number of columns: {len(train.columns)}")
for column in train.columns:
  print(f"{column}: {train[column].nunique()}")


Number of columns: 616
Id: 233234
GameRulesetName: 1377
agent1: 72
agent2: 72
Stochastic: 2
Asymmetric: 2
AsymmetricForces: 2
AsymmetricPiecesType: 2
PlayersWithDirections: 2
Cooperation: 2
Team: 2
Shape: 2
SquareShape: 2
HexShape: 2
TriangleShape: 2
DiamondShape: 2
RectangleShape: 2
SpiralShape: 2
CircleShape: 2
StarShape: 2
SquarePyramidalShape: 2
RegularShape: 2
PolygonShape: 2
Tiling: 2
SquareTiling: 2
HexTiling: 2
TriangleTiling: 2
SemiRegularTiling: 2
MorrisTiling: 2
CircleTiling: 2
ConcentricTiling: 2
SpiralTiling: 2
AlquerqueTiling: 2
MancalaBoard: 2
MancalaStores: 2
MancalaTwoRows: 2
MancalaThreeRows: 2
MancalaFourRows: 2
MancalaSixRows: 2
MancalaCircular: 2
AlquerqueBoard: 2
AlquerqueBoardWithOneTriangle: 2
AlquerqueBoardWithTwoTriangles: 2
AlquerqueBoardWithFourTriangles: 2
AlquerqueBoardWithEightTriangles: 2
ThreeMensMorrisBoard: 2
ThreeMensMorrisBoardWithTwoTriangles: 2
NineMensMorrisBoard: 2
StarBoard: 2
CrossBoard: 2
KintsBoard: 2
PachisiBoard: 2
FortyStonesWithFourGapsB

In [9]:
# prompt: drop all columns where nunique = 0

for column in train.columns:
  if train[column].nunique() == 0:
    train = train.drop(column, axis=1)

print(f"Number of columns: {len(train.columns)}")
for column in train.columns:
  print(f"{column}: {train[column].nunique()}")


Number of columns: 598
Id: 233234
GameRulesetName: 1377
agent1: 72
agent2: 72
Stochastic: 2
Asymmetric: 2
AsymmetricForces: 2
AsymmetricPiecesType: 2
PlayersWithDirections: 2
Cooperation: 2
Team: 2
Shape: 2
SquareShape: 2
HexShape: 2
TriangleShape: 2
DiamondShape: 2
RectangleShape: 2
SpiralShape: 2
CircleShape: 2
StarShape: 2
SquarePyramidalShape: 2
RegularShape: 2
PolygonShape: 2
Tiling: 2
SquareTiling: 2
HexTiling: 2
TriangleTiling: 2
SemiRegularTiling: 2
MorrisTiling: 2
CircleTiling: 2
ConcentricTiling: 2
SpiralTiling: 2
AlquerqueTiling: 2
MancalaBoard: 2
MancalaStores: 2
MancalaTwoRows: 2
MancalaThreeRows: 2
MancalaFourRows: 2
MancalaSixRows: 2
MancalaCircular: 2
AlquerqueBoard: 2
AlquerqueBoardWithOneTriangle: 2
AlquerqueBoardWithTwoTriangles: 2
AlquerqueBoardWithFourTriangles: 2
AlquerqueBoardWithEightTriangles: 2
ThreeMensMorrisBoard: 2
ThreeMensMorrisBoardWithTwoTriangles: 2
NineMensMorrisBoard: 2
StarBoard: 2
CrossBoard: 2
KintsBoard: 2
PachisiBoard: 2
FortyStonesWithFourGapsB

In [10]:
# prompt: print the number of columns and the number of unqiue values of each column

print(f"Number of columns: {len(train.columns)}")
for column in train.columns:
  print(f"{column}: {train[column].nunique()}")



Number of columns: 598
Id: 233234
GameRulesetName: 1377
agent1: 72
agent2: 72
Stochastic: 2
Asymmetric: 2
AsymmetricForces: 2
AsymmetricPiecesType: 2
PlayersWithDirections: 2
Cooperation: 2
Team: 2
Shape: 2
SquareShape: 2
HexShape: 2
TriangleShape: 2
DiamondShape: 2
RectangleShape: 2
SpiralShape: 2
CircleShape: 2
StarShape: 2
SquarePyramidalShape: 2
RegularShape: 2
PolygonShape: 2
Tiling: 2
SquareTiling: 2
HexTiling: 2
TriangleTiling: 2
SemiRegularTiling: 2
MorrisTiling: 2
CircleTiling: 2
ConcentricTiling: 2
SpiralTiling: 2
AlquerqueTiling: 2
MancalaBoard: 2
MancalaStores: 2
MancalaTwoRows: 2
MancalaThreeRows: 2
MancalaFourRows: 2
MancalaSixRows: 2
MancalaCircular: 2
AlquerqueBoard: 2
AlquerqueBoardWithOneTriangle: 2
AlquerqueBoardWithTwoTriangles: 2
AlquerqueBoardWithFourTriangles: 2
AlquerqueBoardWithEightTriangles: 2
ThreeMensMorrisBoard: 2
ThreeMensMorrisBoardWithTwoTriangles: 2
NineMensMorrisBoard: 2
StarBoard: 2
CrossBoard: 2
KintsBoard: 2
PachisiBoard: 2
FortyStonesWithFourGapsB

In [11]:
# prompt: drop Id column

train = train.drop('Id', axis=1)


In [15]:
def data_generator(X, y, batch_size, tfidf_vectorizers, text_features):
    num_samples = len(X)
    while True:
        for i in range(0, num_samples, batch_size):
            batch_indices = slice(i, min(i + batch_size, num_samples))
            X_batch_list = [
                X[numerical_features].iloc[batch_indices].values,
                X[binary_features].iloc[batch_indices].values
            ] + [X[col].iloc[batch_indices].values for col in categorical_features] + get_tfidf_features(X.iloc[batch_indices], tfidf_vectorizers, text_features)
            y_batch = y.iloc[batch_indices]  # y is already y_train or y_val
            yield X_batch_list, y_batch       # Use y_batch directly

KeyError: "['utility_agent1'] not in index"

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate, Embedding, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# ... (Load your data)

# 1. Identify feature types (categorical and text features first)
categorical_features = ['GameRulesetName', 'agent1', 'agent2']
text_features = ['EnglishRules', 'LudRules']

# Save the target variable separately before any transformations
y = train['utility_agent1']

# 2. Encode categorical features
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    label_encoders[col] = le

# 3. Vectorize text features using TF-IDF with memory optimization
tfidf_vectorizers = {}
for col in text_features:
    tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    train_tfidf = tfidf.fit_transform(train[col])
    tfidf_vectorizers[col] = tfidf
    # Ensure unique column names for TF-IDF features
    train_tfidf_df = pd.DataFrame(train_tfidf.toarray(), columns=[f"{col}_{feature}" for feature in tfidf.get_feature_names_out()], index=train.index)
    train = pd.concat([train, train_tfidf_df], axis=1)

# 4. Identify numerical and binary features AFTER TF-IDF concatenation, EXCLUDING TF-IDF columns
# Get the column names for each TF-IDF feature
tfidf_col_names = {col: [c for c in train.columns if c.startswith(col + '_')] for col in text_features}

# Ensure categorical features are not in text_features
categorical_features = [col for col in categorical_features if col not in text_features]

numerical_features = train.select_dtypes(include=['int64', 'float64']).columns.difference(
    [col for col_list in tfidf_col_names.values() for col in col_list] + categorical_features
)
numerical_features = numerical_features[train[numerical_features].nunique() > 2]

binary_features = train.select_dtypes(include=['int64', 'float64']).columns.difference(
    [col for col_list in tfidf_col_names.values() for col in col_list] + categorical_features
)
binary_features = binary_features[train[binary_features].nunique() == 2]

# 5. Scale numerical features
scaler = StandardScaler()
train[numerical_features] = scaler.fit_transform(train[numerical_features])

# 6. Split data into features and labels
# Drop target and original text features
X = train.drop(['utility_agent1'] + text_features, axis=1)

# Now X should only contain numerical, binary, and categorical features
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Build the model

# Input layers
numerical_input = Input(shape=(len(numerical_features),))
binary_input = Input(shape=(len(binary_features),))
categorical_inputs = []
categorical_embeddings = []
text_inputs = []

# Categorical embeddings
for col in categorical_features:
    vocab_size = len(label_encoders[col].classes_) + 1
    embedding_dim = int(min(50, (vocab_size + 1) // 2))
    categorical_input = Input(shape=(1,))
    categorical_inputs.append(categorical_input)
    embedding = Embedding(vocab_size, embedding_dim)(categorical_input)
    embedding_flattened = Flatten()(embedding)
    categorical_embeddings.append(embedding_flattened)

# Text feature inputs
for col in text_features:
    input_dim = len(tfidf_col_names[col])
    text_input = Input(shape=(input_dim,))
    text_inputs.append(text_input)

# Numerical data path
numerical_hidden1 = Dense(64, activation='relu')(numerical_input)
numerical_hidden2 = Dense(32, activation='relu')(numerical_hidden1)

# Concatenate all inputs
merged = Concatenate()([numerical_hidden2, binary_input] + categorical_embeddings + text_inputs)

# Output layer
output = Dense(1)(merged)

# Create the model
model = Model(inputs=[numerical_input, binary_input] + categorical_inputs + text_inputs, outputs=output)

# 8. Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 9. Prepare training and validation data, handling TF-IDF separately

def get_tfidf_features(X, tfidf_vectorizers, text_features):
    return [tfidf_vectorizers[col].transform(X[col]) for col in text_features]

# 10. Fit the model with a generator to load TF-IDF data in batches

def data_generator(X, y, batch_size, tfidf_vectorizers, text_features):
    num_samples = len(X)
    while True:
        for i in range(0, num_samples, batch_size):
            batch_indices = slice(i, min(i + batch_size, num_samples))
            X_batch_list = [
                X[numerical_features].iloc[batch_indices].values,
                X[binary_features].iloc[batch_indices].values
            ] + [X[col].iloc[batch_indices].values for col in categorical_features] + get_tfidf_features(X.iloc[batch_indices], tfidf_vectorizers, text_features)
            y_batch = y.iloc[batch_indices]
            yield X_batch_list, y_batch

batch_size = 32
train_generator = data_generator(X_train, y_train, batch_size, tfidf_vectorizers, text_features)
val_generator = data_generator(X_val, y_val, batch_size, tfidf_vectorizers, text_features)

steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

In [21]:
history = model.fit(
    train_generator,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

# 11. Evaluate the model performance
# Prepare validation data again (outside the generator) to evaluate

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate, Embedding, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have your 'train' DataFrame ready
# For example, you might load your data like this:
# train = pd.read_csv('your_dataset.csv')

# 1. Identify feature types (categorical and text features first)
categorical_features = ['GameRulesetName', 'agent1', 'agent2']
text_features = ['EnglishRules', 'LudRules']

# Save the target variable separately before any transformations
y = train['utility_agent1']

# 2. Encode categorical features
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    label_encoders[col] = le

# 3